In [7]:
import numpy as np 
import pandas as pd 
import torch 


In [8]:
df = pd.read_csv("bwt.csv")

In [10]:
df.head()

,Unnamed: 0,bwt,gestation,parity,age,height,weight,smoke
0,1,120,284,0,27,62,100,0
1,2,113,282,0,33,64,135,0
2,3,128,279,0,28,64,115,1
3,4,108,282,0,23,67,125,1
4,5,136,286,0,25,62,93,0


In [33]:
inputs = np.array(df[['gestation','parity','age','height','weight','smoke']],dtype='float32')

In [34]:
targets = np.array(df['bwt'],dtype='float32').reshape(-1,1)

In [35]:
inputq = torch.from_numpy(inputs)
targetq = torch.from_numpy(targets)

In [36]:
targetq.shape

torch.Size([1174, 1])

In [37]:
inputq.shape

torch.Size([1174, 6])

In [38]:
from torch.utils.data import TensorDataset,DataLoader

In [59]:
train_ds = TensorDataset(inputq,targetq)
train_ds[0:75]

(tensor([[284.,   0.,  27.,  62., 100.,   0.],
         [282.,   0.,  33.,  64., 135.,   0.],
         [279.,   0.,  28.,  64., 115.,   1.],
         [282.,   0.,  23.,  67., 125.,   1.],
         [286.,   0.,  25.,  62.,  93.,   0.],
         [244.,   0.,  33.,  62., 178.,   0.],
         [245.,   0.,  23.,  65., 140.,   0.],
         [289.,   0.,  25.,  62., 125.,   0.],
         [299.,   0.,  30.,  66., 136.,   1.],
         [351.,   0.,  27.,  68., 120.,   0.],
         [282.,   0.,  32.,  64., 124.,   1.],
         [279.,   0.,  23.,  63., 128.,   1.],
         [281.,   0.,  36.,  61.,  99.,   1.],
         [273.,   0.,  30.,  63., 154.,   0.],
         [285.,   0.,  38.,  63., 130.,   0.],
         [255.,   0.,  25.,  65., 125.,   1.],
         [261.,   0.,  33.,  60., 125.,   1.],
         [261.,   0.,  33.,  68., 170.,   0.],
         [288.,   0.,  43.,  66., 142.,   1.],
         [270.,   0.,  22.,  56.,  93.,   0.],
         [274.,   0.,  27.,  67., 175.,   1.],
         [287

In [162]:
batch_size = 1
train_dl = DataLoader(train_ds , batch_size , shuffle = True )

In [163]:
import torch.nn as nn 

In [164]:
model = nn.Linear(6,1)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3665, -0.1225, -0.2309, -0.2298, -0.3123, -0.1132]],
       requires_grad=True)
Parameter containing:
tensor([0.1460], requires_grad=True)


In [165]:
preds = model(inputq)
preds

tensor([[-155.6422],
        [-167.6841],
        [-159.2978],
        ...,
        [-175.3170],
        [-157.0923],
        [-172.6916]], grad_fn=<AddmmBackward>)

In [166]:
targetq

tensor([[120.],
        [113.],
        [128.],
        ...,
        [130.],
        [125.],
        [117.]])

In [167]:
import torch.nn.functional as F 

In [168]:
loss_fn = F.mse_loss

In [169]:
loss = loss_fn(model(inputq),targetq)
loss

tensor(80528.3828, grad_fn=<MseLossBackward>)

In [170]:
opt = torch.optim.Adamax(model.parameters(), lr=1e-5)

In [172]:
def fit(epochs,model,loss_fn,opt,train_dl):
    for ep in range(epochs):
        for x,y in train_dl:
            preds = model(x)
            
            loss= loss_fn(preds,y)
            
            loss.backward()
            
            opt.step()
            
            opt.zero_grad()
            
        if (ep+1)%100== 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(ep+1,epochs,loss.item()))

In [173]:
fit(400,model,loss_fn,opt,train_dl)

Epoch [100/400], Loss: 0.7631
Epoch [200/400], Loss: 89.6877
Epoch [300/400], Loss: 63.4647
Epoch [400/400], Loss: 2.1150


In [178]:
preds = model(inputq)
preds

tensor([[116.0765],
        [122.5396],
        [117.8489],
        ...,
        [126.4828],
        [116.4610],
        [127.0590]], grad_fn=<AddmmBackward>)

In [177]:
targetq

tensor([[120.],
        [113.],
        [128.],
        ...,
        [130.],
        [125.],
        [117.]])